# **Getting Open Critic data, from API to CSV.**

This notebook use Open Critic public API to get the Hall of Fame gamesh along the years, transform to the correct format and save them into a CSV.

My main goals here were:
* Learn how to use request library to get API data
* learn how to work with Json responses and how convert to DataFrame in the correct formats



In [2]:
import requests
import json
import pandas as pd
import datetime as dt

In [3]:
# Function to get multiple responses of a REST API.
# The function get a year range and return a list of "hall of fame" games over years
# The Endpoint used here accept just a year as parameter (not a range), this is why the function was necessary

def get_response(init_year, end_year):
    years = range(init_year, end_year+1, 1)
    response_list=[]
    
    for year in years:

        url = "https://opencritic-api.p.rapidapi.com/game/hall-of-fame/" + str(year)
        headers = {
            "X-RapidAPI-Key": "589a49664fmshb31addec27c7813p100adbjsn8ca39eb71bf4",
            "X-RapidAPI-Host": "opencritic-api.p.rapidapi.com"
        }
        # Using the request method to GET the response
        response = requests.request('GET', url, headers=headers)
        # Append the multiple responses
        response_list.append(response.json())
    return response_list
    

In [4]:
# This function extract only de feature needed from the Json format response list

def get_feature_list(feature1,feature2, feature3):
    
    list1=[]
    list2=[]
    list3=[]
    feature_list = []
    #Get the response list of desired year range
    response_json=get_response(2016,2022)
    
    # response_json is a list of lists, so here I flattened it
    flatten_response_json=[ite for sub_list in response_json for ite in sub_list]

    
    for item in range(len(flatten_response_json)):
        aux1 = flatten_response_json[item][feature1]
        aux2 = flatten_response_json[item][feature2]
        aux3 = flatten_response_json[item][feature3]
        
        list1.append(aux1)
        list2.append(aux2)
        list3.append(aux3)
        
    feature_list.append(list1)
    feature_list.append(list2)
    feature_list.append(list3)

    return feature_list

In [5]:
data = get_feature_list('name','firstReleaseDate','topCriticScore')
df_OpenCritic = pd.DataFrame(data).transpose()

columns=['name','firstReleaseDate','topCriticScore']
df_OpenCritic.columns=columns

df_OpenCritic

,name,firstReleaseDate,topCriticScore
0,Uncharted 4: A Thief's End,2016-05-10T00:00:00.000Z,93
1,INSIDE,2016-06-29T00:00:00.000Z,91
2,Forza Horizon 3,2016-09-27T00:00:00.000Z,91
3,Overwatch,2016-05-24T00:00:00.000Z,90
4,Dark Souls III,2016-04-12T00:00:00.000Z,90
...,...,...,...
79,Rogue Legacy 2,2022-04-28T00:00:00.000Z,89
80,The Case of the Golden Idol,2022-10-13T00:00:00.000Z,88
81,Immortality,2022-08-30T00:00:00.000Z,88
82,Horizon Forbidden West,2022-02-18T00:00:00.000Z,88


In [6]:
df_OpenCritic['firstReleaseDate'] = df_OpenCritic['firstReleaseDate'].str[:-14]
df_OpenCritic['firstReleaseDate'] = pd.to_datetime(df_OpenCritic['firstReleaseDate'])
df_OpenCritic['topCriticScore'] = df_OpenCritic['topCriticScore'].astype(int)
df_OpenCritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   name              84 non-null     object        
 1   firstReleaseDate  84 non-null     datetime64[ns]
 2   topCriticScore    84 non-null     int32         
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 1.8+ KB


In [7]:
df_OpenCritic.head(5)

,name,firstReleaseDate,topCriticScore
0,Uncharted 4: A Thief's End,2016-05-10,93
1,INSIDE,2016-06-29,91
2,Forza Horizon 3,2016-09-27,91
3,Overwatch,2016-05-24,90
4,Dark Souls III,2016-04-12,90


In [8]:
df_OpenCritic.to_csv('hall_of_fame.csv', index=None)